In [2]:
# Load songs, albums, and songs+albums csvs

import pandas as pd

# Load cleaned songs dataframe
df_songs = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_cleaned.csv')

# Load cleaned albums dataframe
df_albums = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_cleaned.csv')

# Load combined dataframe
df_combined = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_songs_and_albums_combined_all.csv')

# Display info
print("✅ Loaded all dataframes")
print(f"\ndf_songs: {df_songs.shape[0]:,} rows, {df_songs.shape[1]} columns")
print(f"df_albums: {df_albums.shape[0]:,} rows, {df_albums.shape[1]} columns")
print(f"df_combined: {df_combined.shape[0]:,} rows, {df_combined.shape[1]} columns")


✅ Loaded all dataframes

df_songs: 37,337 rows, 7 columns
df_albums: 38,823 rows, 7 columns
df_combined: 76,160 rows, 8 columns


In [3]:
# Filter for songs that reached top 10 (peak_pos <= 10)
df_top10_songs = df_songs[df_songs['peak_pos'] <= 10]

# Get unique performers with top 10 hits
top10_artists = df_top10_songs['performer_normalized'].unique()

print("="*60)
print("TOP 10 ARTISTS ANALYSIS")
print("="*60)
print(f"Unique artists with at least one top 10 song: {len(top10_artists):,}")
print(f"Total top 10 song entries: {len(df_top10_songs):,}")
print(f"Total unique artists in df_songs: {df_songs['performer_normalized'].nunique():,}")
print(f"Percentage with top 10 hit: {len(top10_artists) / df_songs['performer_normalized'].nunique() * 100:.1f}%")
print("="*60)

# Show top artists by number of top 10 hits
print("\nTop 10 artists by number of top 10 hits:")
top10_counts = df_top10_songs['performer_normalized'].value_counts().head(10)
print(top10_counts)


TOP 10 ARTISTS ANALYSIS
Unique artists with at least one top 10 song: 2,467
Total top 10 song entries: 6,057
Total unique artists in df_songs: 9,429
Percentage with top 10 hit: 26.2%

Top 10 artists by number of top 10 hits:
performer_normalized
drake              75
taylor swift       69
madonna            38
the beatles        35
rihanna            31
michael jackson    30
mariah carey       27
stevie wonder      26
elton john         26
elvis presley      25
Name: count, dtype: int64


In [4]:
# Count top 10 songs per artist
top10_counts = df_top10_songs['performer_normalized'].value_counts()

# Filter for artists with exactly 1 top 10 song
one_hit_wonders_top10 = top10_counts[top10_counts == 1]

print("="*60)
print("ONE-HIT WONDERS (TOP 10)")
print("="*60)
print(f"Artists with ONLY ONE top 10 song: {len(one_hit_wonders_top10):,}")
print(f"Artists with 2+ top 10 songs: {len(top10_counts[top10_counts > 1]):,}")
print(f"Total artists with top 10 hits: {len(top10_counts):,}")
print(f"\nPercentage with only 1 top 10 hit: {len(one_hit_wonders_top10) / len(top10_counts) * 100:.1f}%")
print("="*60)

# Show some examples of one-hit wonders
print("\nSample one-hit wonders (top 10):")
print(one_hit_wonders_top10.head(20))

# Show their song details
print("\nDetails of some one-hit wonders:")
sample_artists = one_hit_wonders_top10.head(10).index
sample_songs = df_top10_songs[df_top10_songs['performer_normalized'].isin(sample_artists)]
print(sample_songs[['performer', 'title', 'peak_pos', 'wks_on_chart', 'first_charting_year']].to_string(index=False))


ONE-HIT WONDERS (TOP 10)
Artists with ONLY ONE top 10 song: 1,531
Artists with 2+ top 10 songs: 936
Total artists with top 10 hits: 2,467

Percentage with only 1 top 10 hit: 62.1%

Sample one-hit wonders (top 10):
performer_normalized
shawn mendes & justin bieber    1
paula cole                      1
juice wrld & marshmello         1
shawn colvin                    1
polo g & the kid laroi          1
polow da don                    1
lil mosey                       1
21 savage & project pat         1
21 savage & metro boomin        1
foxy brown                      1
gayle                           1
lord tariq & peter gunz         1
don toliver & nav               1
ajr                             1
jid                             1
buckcherry                      1
masked wolf                     1
b-rock & the bizz               1
internet money & gunna          1
lil baby & dababy               1
Name: count, dtype: int64

Details of some one-hit wonders:
                   perfor

In [6]:
import requests
import time
from urllib.parse import quote

def check_musicbrainz(artist_name, max_retries=2):
    """Check if artist exists in MusicBrainz"""
    base_url = "https://musicbrainz.org/ws/2/artist/"
    encoded_name = quote(artist_name)
    url = f"{base_url}?query=artist:{encoded_name}&fmt=json"
    
    for attempt in range(max_retries):
        try:
            headers = {'User-Agent': 'BillboardDataCleaner/1.0 (educational-project)'}
            response = requests.get(url, headers=headers, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                if 'artists' in data and len(data['artists']) > 0:
                    for artist in data['artists']:
                        if artist['name'].lower() == artist_name.lower():
                            return True, 100
                        if 'aliases' in artist:
                            for alias in artist['aliases']:
                                if alias.get('name', '').lower() == artist_name.lower():
                                    return True, 100
                    return False, 0
                else:
                    return False, 0
            elif response.status_code == 503:
                time.sleep(3)
                continue
            else:
                return False, 0
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            return False, 0
    
    return False, 0

def check_musicbrainz_variants(artist_name, max_retries=2):
    """Check with & and 'and' variants"""
    exists, confidence = check_musicbrainz(artist_name, max_retries)
    if exists:
        return exists, confidence
    
    if '&' in artist_name:
        variant = artist_name.replace('&', 'and')
        exists, confidence = check_musicbrainz(variant, max_retries)
        if exists:
            return exists, confidence
    elif ' and ' in artist_name.lower():
        variant = re.sub(r'\s+[Aa]nd\s+', ' & ', artist_name)
        exists, confidence = check_musicbrainz(variant, max_retries)
        if exists:
            return exists, confidence
    
    return False, 0

# Now run the categorization
one_hit_artists_list = one_hit_wonders_top10.index.tolist()

collaboration_indicators = r'&|[Ff]eat|[Ff]t\.|[Ww]ith|X|,'

artists_with_indicators = [
    artist for artist in one_hit_artists_list 
    if pd.Series([artist]).str.contains(collaboration_indicators, regex=True).iloc[0]
]

artists_without_indicators = [
    artist for artist in one_hit_artists_list 
    if artist not in artists_with_indicators
]

print(f"Checking {len(artists_with_indicators)} artists with collaboration indicators against MusicBrainz...")
print("This will take a few minutes...\n")

verified_bands = []
actual_collaborations = []

for i, artist in enumerate(artists_with_indicators):
    if i % 10 == 0:
        print(f"Progress: {i}/{len(artists_with_indicators)}")
    
    is_verified, confidence = check_musicbrainz_variants(artist)
    
    if is_verified:
        verified_bands.append(artist)
    else:
        actual_collaborations.append(artist)
    
    time.sleep(1.5)

print("\n" + "="*60)
print("ONE-HIT WONDERS CATEGORIZATION")
print("="*60)
print(f"Total one-hit wonders: {len(one_hit_artists_list):,}")
print(f"\nBreakdown:")
print(f"  Solo/single group (no '&'/'and'): {len(artists_without_indicators):,}")
print(f"  Real bands (has '&'/'and', verified in MusicBrainz): {len(verified_bands):,}")
print(f"  Actual collaborations (has '&'/'and', NOT in MusicBrainz): {len(actual_collaborations):,}")
print(f"\nTrue solo artists/single groups: {len(artists_without_indicators) + len(verified_bands):,}")
print(f"Collaborations: {len(actual_collaborations):,}")
print("="*60)

# Show examples
print("\nVerified bands with '&' in name:")
for artist in verified_bands[:10]:
    song = df_top10_songs[df_top10_songs['performer_normalized'] == artist].iloc[0]
    print(f"  - {song['performer']} - '{song['title']}'")

print("\nActual collaborations (not verified as single artist):")
for artist in actual_collaborations[:10]:
    song = df_top10_songs[df_top10_songs['performer_normalized'] == artist].iloc[0]
    print(f"  - {song['performer']} - '{song['title']}'")


Checking 184 artists with collaboration indicators against MusicBrainz...
This will take a few minutes...

Progress: 0/184
Progress: 10/184
Progress: 20/184
Progress: 30/184
Progress: 40/184
Progress: 50/184
Progress: 60/184
Progress: 70/184
Progress: 80/184
Progress: 90/184
Progress: 100/184
Progress: 110/184
Progress: 120/184
Progress: 130/184
Progress: 140/184
Progress: 150/184
Progress: 160/184
Progress: 170/184
Progress: 180/184

ONE-HIT WONDERS CATEGORIZATION
Total one-hit wonders: 1,531

Breakdown:
  Solo/single group (no '&'/'and'): 1,347
  Real bands (has '&'/'and', verified in MusicBrainz): 64
  Actual collaborations (has '&'/'and', NOT in MusicBrainz): 120

True solo artists/single groups: 1,411
Collaborations: 120

Verified bands with '&' in name:
  - Shawn Mendes & Justin Bieber - 'Monster'
  - Lord Tariq & Peter Gunz - 'Deja Vu [Uptown Baby]'
  - B-Rock & The Bizz - 'My Baby Daddy'
  - Trina & Tamara - 'My Love Is The Shhh!'
  - K.P. & Envyi - 'Swing My Way'
  - Lady Gaga

In [8]:
import re

# Get all known performers
all_known_performers = set(df_combined['performer_normalized'].unique())

print(f"Total known performers in combined dataframe: {len(all_known_performers):,}")

# Load existing cache if available
import os
cache_path = os.path.expanduser('~/artist_verification_cache.csv')
try:
    cache_df = pd.read_csv(cache_path)
    musicbrainz_cache = dict(zip(cache_df['artist'], cache_df['is_real_artist']))
    print(f"Loaded {len(musicbrainz_cache):,} entries from existing cache")
except:
    musicbrainz_cache = {}

print("Using known performers + cache only (no new API calls)\n")

def split_if_verified_fast(performer, known_performers, mb_cache):
    """Split performer if verified in known_performers OR cache"""
    # Check for collaboration indicators
    if not re.search(r'&|[Aa]nd|[Ff]eat|[Ff]t\.|[Ww]ith|X|,', performer):
        return [performer]
    
    # Try different split patterns
    performers_list = None
    
    if re.search(r'\s+[Ff]eatur(ing|e)\s+', performer) or re.search(r'\s+[Ff]t\.?\s+', performer):
        performers_list = re.split(r'\s+[Ff]eatur(?:ing|e)\s+|\s+[Ff]t\.?\s+', performer)
    elif re.search(r'\s+[Ww]ith\s+', performer):
        performers_list = re.split(r'\s+[Ww]ith\s+', performer)
    elif ' X ' in performer or ' x ' in performer:
        performers_list = re.split(r'\s+[Xx]\s+', performer)
    elif ',' in performer:
        parts = [p.strip() for p in performer.split(',')]
        if len(parts) > 1 and (' & ' in parts[-1] or ' And ' in parts[-1] or ' and ' in parts[-1]):
            last_part_split = re.split(r'\s+&\s+|\s+[Aa]nd\s+', parts[-1])
            performers_list = parts[:-1] + last_part_split
        else:
            performers_list = parts
    elif ' & ' in performer or ' And ' in performer or ' and ' in performer:
        performers_list = re.split(r'\s+&\s+|\s+[Aa]nd\s+', performer)
    
    if not performers_list or len(performers_list) <= 1:
        return [performer]
    
    # Clean up
    performers_list = [p.strip() for p in performers_list if p.strip()]
    
    # Check each part in known_performers OR cache
    verified_parts = []
    for part in performers_list:
        if part in known_performers:
            verified_parts.append(True)
        elif part in mb_cache and mb_cache[part]:
            verified_parts.append(True)
        else:
            verified_parts.append(False)
    
    # If ALL parts verified, split
    if all(verified_parts):
        return performers_list
    else:
        return [performer]

# Process SONGS
print("Processing SONGS dataframe...")
songs_rows = []
splits_count = 0

for idx, row in df_songs.iterrows():
    if idx % 5000 == 0:
        print(f"  Progress: {idx:,}/{len(df_songs):,}")
    
    performer = row['performer_normalized']
    split_performers = split_if_verified_fast(performer, all_known_performers, musicbrainz_cache)
    
    if len(split_performers) > 1:
        splits_count += 1
        for split_perf in split_performers:
            new_row = row.copy()
            new_row['performer_normalized'] = split_perf
            new_row['original_performer_normalized'] = performer
            songs_rows.append(new_row)
    else:
        songs_rows.append(row)

df_songs_resplit = pd.DataFrame(songs_rows)

print(f"\nSONGS Results:")
print(f"  Original rows: {len(df_songs):,}")
print(f"  New rows: {len(df_songs_resplit):,}")
print(f"  Performers split: {splits_count:,}")

# Process ALBUMS
print("\nProcessing ALBUMS dataframe...")
albums_rows = []
splits_count = 0

for idx, row in df_albums.iterrows():
    if idx % 5000 == 0:
        print(f"  Progress: {idx:,}/{len(df_albums):,}")
    
    performer = row['performer_normalized']
    split_performers = split_if_verified_fast(performer, all_known_performers, musicbrainz_cache)
    
    if len(split_performers) > 1:
        splits_count += 1
        for split_perf in split_performers:
            new_row = row.copy()
            new_row['performer_normalized'] = split_perf
            new_row['original_performer_normalized'] = performer
            albums_rows.append(new_row)
    else:
        albums_rows.append(row)

df_albums_resplit = pd.DataFrame(albums_rows)

print(f"\nALBUMS Results:")
print(f"  Original rows: {len(df_albums):,}")
print(f"  New rows: {len(df_albums_resplit):,}")
print(f"  Performers split: {splits_count:,}")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Songs: {len(df_songs):,} → {len(df_songs_resplit):,} rows")
print(f"Albums: {len(df_albums):,} → {len(df_albums_resplit):,} rows")
print("="*60)


Total known performers in combined dataframe: 14,864
Loaded 10,008 entries from existing cache
Using known performers + cache only (no new API calls)

Processing SONGS dataframe...
  Progress: 0/37,337
  Progress: 5,000/37,337
  Progress: 10,000/37,337
  Progress: 15,000/37,337
  Progress: 20,000/37,337
  Progress: 25,000/37,337
  Progress: 30,000/37,337
  Progress: 35,000/37,337

SONGS Results:
  Original rows: 37,337
  New rows: 38,383
  Performers split: 904

Processing ALBUMS dataframe...
  Progress: 0/38,823
  Progress: 5,000/38,823
  Progress: 10,000/38,823
  Progress: 15,000/38,823
  Progress: 20,000/38,823
  Progress: 25,000/38,823
  Progress: 30,000/38,823
  Progress: 35,000/38,823

ALBUMS Results:
  Original rows: 38,823
  New rows: 39,129
  Performers split: 302

SUMMARY
Songs: 37,337 → 38,383 rows
Albums: 38,823 → 39,129 rows


In [9]:
# Check SONGS resplit dataframe
songs_with_performer = df_songs_resplit['performer_normalized'].notna().sum()
songs_total = len(df_songs_resplit)

print("="*60)
print("PERFORMER_NORMALIZED DATA CHECK")
print("="*60)

print("\nSONGS (df_songs_resplit):")
print(f"  Total rows: {songs_total:,}")
print(f"  Rows with performer_normalized: {songs_with_performer:,}")
print(f"  Rows with null/missing: {songs_total - songs_with_performer:,}")
print(f"  Percentage complete: {songs_with_performer / songs_total * 100:.2f}%")

# Check ALBUMS resplit dataframe
albums_with_performer = df_albums_resplit['performer_normalized'].notna().sum()
albums_total = len(df_albums_resplit)

print("\nALBUMS (df_albums_resplit):")
print(f"  Total rows: {albums_total:,}")
print(f"  Rows with performer_normalized: {albums_with_performer:,}")
print(f"  Rows with null/missing: {albums_total - albums_with_performer:,}")
print(f"  Percentage complete: {albums_with_performer / albums_total * 100:.2f}%")

print("\n" + "="*60)

# Show sample of any null values if they exist
if songs_total - songs_with_performer > 0:
    print("\nSample songs rows with missing performer_normalized:")
    print(df_songs_resplit[df_songs_resplit['performer_normalized'].isna()].head())

if albums_total - albums_with_performer > 0:
    print("\nSample albums rows with missing performer_normalized:")
    print(df_albums_resplit[df_albums_resplit['performer_normalized'].isna()].head())


PERFORMER_NORMALIZED DATA CHECK

SONGS (df_songs_resplit):
  Total rows: 38,383
  Rows with performer_normalized: 38,383
  Rows with null/missing: 0
  Percentage complete: 100.00%

ALBUMS (df_albums_resplit):
  Total rows: 39,129
  Rows with performer_normalized: 39,129
  Rows with null/missing: 0
  Percentage complete: 100.00%



In [10]:
# Save resplit songs dataframe
songs_resplit_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_final.csv'
df_songs_resplit.to_csv(songs_resplit_path, index=False)

print("✅ Saved resplit SONGS dataframe")
print(f"📁 File: {songs_resplit_path}")
print(f"📊 Rows: {len(df_songs_resplit):,}")
print(f"🎤 Unique performers: {df_songs_resplit['performer_normalized'].nunique():,}")

print("\n" + "="*60 + "\n")

# Save resplit albums dataframe
albums_resplit_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_final.csv'
df_albums_resplit.to_csv(albums_resplit_path, index=False)

print("✅ Saved resplit ALBUMS dataframe")
print(f"📁 File: {albums_resplit_path}")
print(f"📊 Rows: {len(df_albums_resplit):,}")
print(f"🎤 Unique performers: {df_albums_resplit['performer_normalized'].nunique():,}")

print("\n" + "="*60)
print("Both final dataframes saved! 🎉")
print("="*60)


✅ Saved resplit SONGS dataframe
📁 File: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_final.csv
📊 Rows: 38,383
🎤 Unique performers: 8,903


✅ Saved resplit ALBUMS dataframe
📁 File: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_final.csv
📊 Rows: 39,129
🎤 Unique performers: 9,977

Both final dataframes saved! 🎉


In [11]:
import pandas as pd

# Load final songs dataframe
df_songs = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_final.csv')

# Load final albums dataframe
df_albums = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_final.csv')

# Display info
print("✅ Loaded final dataframes")
print(f"\ndf_songs: {df_songs.shape[0]:,} rows, {df_songs.shape[1]} columns")
print(f"df_albums: {df_albums.shape[0]:,} rows, {df_albums.shape[1]} columns")
print(f"\nUnique performers:")
print(f"  Songs: {df_songs['performer_normalized'].nunique():,}")
print(f"  Albums: {df_albums['performer_normalized'].nunique():,}")


✅ Loaded final dataframes

df_songs: 38,383 rows, 8 columns
df_albums: 39,129 rows, 8 columns

Unique performers:
  Songs: 8,903
  Albums: 9,977


In [12]:
# Filter for songs that reached top 10 (peak_pos <= 10)
df_top10_songs = df_songs[df_songs['peak_pos'] <= 10]

# Get unique performers with top 10 hits
top10_artists = df_top10_songs['performer_normalized'].unique()

print("="*60)
print("TOP 10 ARTISTS ANALYSIS")
print("="*60)
print(f"Unique artists with at least one top 10 song: {len(top10_artists):,}")
print(f"Total top 10 song entries: {len(df_top10_songs):,}")
print(f"Total unique artists in df_songs: {df_songs['performer_normalized'].nunique():,}")
print(f"Percentage with top 10 hit: {len(top10_artists) / df_songs['performer_normalized'].nunique() * 100:.1f}%")
print("="*60)

# Count top 10 songs per artist
top10_counts = df_top10_songs['performer_normalized'].value_counts()

# Filter for artists with exactly 1 top 10 song
one_hit_wonders_top10 = top10_counts[top10_counts == 1]

print("\n" + "="*60)
print("ONE-HIT WONDERS (TOP 10)")
print("="*60)
print(f"Artists with ONLY ONE top 10 song: {len(one_hit_wonders_top10):,}")
print(f"Artists with 2+ top 10 songs: {len(top10_counts[top10_counts > 1]):,}")
print(f"Total artists with top 10 hits: {len(top10_counts):,}")
print(f"\nPercentage with only 1 top 10 hit: {len(one_hit_wonders_top10) / len(top10_counts) * 100:.1f}%")
print("="*60)

# Show top artists by number of top 10 hits
print("\nTop 15 artists by number of top 10 hits:")
print(top10_counts.head(15))

# Show some one-hit wonders
print("\nSample one-hit wonders (top 10):")
sample_one_hits = one_hit_wonders_top10.head(20)
for artist in sample_one_hits.index:
    song = df_top10_songs[df_top10_songs['performer_normalized'] == artist].iloc[0]
    print(f"  {artist}: '{song['title']}' (peak: {song['peak_pos']}, {song['first_charting_year']})")


TOP 10 ARTISTS ANALYSIS
Unique artists with at least one top 10 song: 2,420
Total top 10 song entries: 6,221
Total unique artists in df_songs: 8,903
Percentage with top 10 hit: 27.2%

ONE-HIT WONDERS (TOP 10)
Artists with ONLY ONE top 10 song: 1,467
Artists with 2+ top 10 songs: 953
Total artists with top 10 hits: 2,420

Percentage with only 1 top 10 hit: 60.6%

Top 15 artists by number of top 10 hits:
performer_normalized
drake                 81
taylor swift          69
madonna               38
the beatles           35
rihanna               32
michael jackson       30
mariah carey          28
justin bieber         27
elton john            26
stevie wonder         26
elvis presley         25
lil wayne             25
kendrick lamar        23
the rolling stones    23
eminem                23
Name: count, dtype: int64

Sample one-hit wonders (top 10):
  estelle: 'American Boy' (peak: 9, 2008)
  allure: 'All Cried Out' (peak: 4, 1997)
  mikky ekko: 'Stay' (peak: 3, 2013)
  chencho corleon

In [13]:
# Filter for albums that reached top 10 (peak_pos <= 10)
df_top10_albums = df_albums[df_albums['peak_pos'] <= 10]

# Get unique performers with top 10 hits
top10_album_artists = df_top10_albums['performer_normalized'].unique()

print("="*60)
print("TOP 10 ALBUM ARTISTS ANALYSIS")
print("="*60)
print(f"Unique artists with at least one top 10 album: {len(top10_album_artists):,}")
print(f"Total top 10 album entries: {len(df_top10_albums):,}")
print(f"Total unique artists in df_albums: {df_albums['performer_normalized'].nunique():,}")
print(f"Percentage with top 10 album: {len(top10_album_artists) / df_albums['performer_normalized'].nunique() * 100:.1f}%")
print("="*60)

# Count top 10 albums per artist
top10_album_counts = df_top10_albums['performer_normalized'].value_counts()

# Filter for artists with exactly 1 top 10 album
one_album_wonders_top10 = top10_album_counts[top10_album_counts == 1]

print("\n" + "="*60)
print("ONE-ALBUM WONDERS (TOP 10)")
print("="*60)
print(f"Artists with ONLY ONE top 10 album: {len(one_album_wonders_top10):,}")
print(f"Artists with 2+ top 10 albums: {len(top10_album_counts[top10_album_counts > 1]):,}")
print(f"Total artists with top 10 albums: {len(top10_album_counts):,}")
print(f"\nPercentage with only 1 top 10 album: {len(one_album_wonders_top10) / len(top10_album_counts) * 100:.1f}%")
print("="*60)

# Show top artists by number of top 10 albums
print("\nTop 15 artists by number of top 10 albums:")
print(top10_album_counts.head(15))

# Show some one-album wonders
print("\nSample one-album wonders (top 10):")
sample_one_albums = one_album_wonders_top10.head(20)
for artist in sample_one_albums.index:
    album = df_top10_albums[df_top10_albums['performer_normalized'] == artist].iloc[0]
    print(f"  {artist}: '{album['title']}' (peak: {album['peak_pos']}, {album['first_charting_year']})")


TOP 10 ALBUM ARTISTS ANALYSIS
Unique artists with at least one top 10 album: 1,868
Total top 10 album entries: 5,908
Total unique artists in df_albums: 9,977
Percentage with top 10 album: 18.7%

ONE-ALBUM WONDERS (TOP 10)
Artists with ONLY ONE top 10 album: 843
Artists with 2+ top 10 albums: 1,025
Total artists with top 10 albums: 1,868

Percentage with only 1 top 10 album: 45.1%

Top 15 artists by number of top 10 albums:
performer_normalized
soundtrack            240
various artists       149
the rolling stones     33
barbra streisand       26
the beatles            26
kidz bop kids          24
madonna                21
bob dylan              21
elton john             21
george strait          21
bruce springsteen      20
mariah carey           19
garth brooks           19
taylor swift           19
future                 18
Name: count, dtype: int64

Sample one-album wonders (top 10):
  ethel cain: 'Preacher's Daughter' (peak: 10, 2025)
  all that remains: 'For We Are Many' (peak: 10

In [15]:
import pandas as pd

# Function to analyze at different thresholds
def analyze_thresholds(df, thresholds=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]):
    results = []
    
    for threshold in thresholds:
        # Filter for songs/albums at this threshold
        df_threshold = df[df['peak_pos'] <= threshold]
        
        # Count per artist
        counts = df_threshold['performer_normalized'].value_counts()
        
        # One-hit wonders
        one_hitters = counts[counts == 1]
        
        results.append({
            'Top': threshold,
            'Total Artists': len(counts),
            'One-Hit Wonders': len(one_hitters),
            '% One-Hit': f"{len(one_hitters) / len(counts) * 100:.1f}%"
        })
    
    return pd.DataFrame(results)

print("="*60)
print("SONGS - One-Hit Wonder Analysis by Chart Position")
print("="*60)
songs_analysis = analyze_thresholds(df_songs)
print(songs_analysis.to_string(index=False))

print("\n" + "="*60)
print("ALBUMS - One-Album Wonder Analysis by Chart Position")
print("="*60)
albums_analysis = analyze_thresholds(df_albums)
print(albums_analysis.to_string(index=False))

print("\n" + "="*60)
print("COMPARISON")
print("="*60)
comparison = pd.DataFrame({
    'Top': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'Songs - Total': songs_analysis['Total Artists'].values,
    'Songs - 1-Hit': songs_analysis['One-Hit Wonders'].values,
    'Songs - %': songs_analysis['% One-Hit'].values,
    'Albums - Total': albums_analysis['Total Artists'].values,
    'Albums - 1-Album': albums_analysis['One-Hit Wonders'].values,
    'Albums - %': albums_analysis['% One-Hit'].values
})
print(comparison.to_string(index=False))


SONGS - One-Hit Wonder Analysis by Chart Position
 Top  Total Artists  One-Hit Wonders % One-Hit
  10           2420             1467     60.6%
  20           3344             1958     58.6%
  30           4131             2358     57.1%
  40           4788             2681     56.0%
  50           5405             2948     54.5%
  60           6096             3283     53.9%
  70           6787             3602     53.1%
  80           7441             3918     52.7%
  90           8181             4304     52.6%
 100           8903             4700     52.8%

ALBUMS - One-Album Wonder Analysis by Chart Position
 Top  Total Artists  One-Hit Wonders % One-Hit
  10           1868              843     45.1%
  20           2641             1124     42.6%
  30           3326             1420     42.7%
  40           3895             1641     42.1%
  50           4429             1879     42.4%
  60           4894             2070     42.3%
  70           5324             2254     42.3%
  8

In [16]:
# Check song title data quality
print("="*60)
print("SONG TITLE ANALYSIS")
print("="*60)

# Sample titles
print("\nSample titles:")
print(df_songs['title'].head(20).to_list())

# Check for common issues
print(f"\nTotal songs: {len(df_songs):,}")
print(f"Unique titles: {df_songs['title'].nunique():,}")

# Check for potential issues
issues = {
    'Leading/trailing spaces': df_songs['title'].str.strip().ne(df_songs['title']).sum(),
    'Multiple spaces': df_songs['title'].str.contains(r'  +', regex=True, na=False).sum(),
    'Parentheses (features/versions)': df_songs['title'].str.contains(r'\(', na=False).sum(),
    'Brackets': df_songs['title'].str.contains(r'\[', na=False).sum(),
    'Quotation marks': df_songs['title'].str.contains(r'["""'']', regex=True, na=False).sum(),
    'All caps': df_songs['title'].str.isupper().sum(),
    'Special chars (non-ASCII)': df_songs['title'].str.contains(r'[^\x00-\x7F]', regex=True, na=False).sum(),
}

print("\nPotential issues found:")
for issue, count in issues.items():
    print(f"  {issue}: {count:,} ({count/len(df_songs)*100:.1f}%)")

# Check for parenthetical content
titles_with_parens = df_songs[df_songs['title'].str.contains(r'\(', na=False)]['title'].head(20)
print("\nSample titles with parentheses:")
for title in titles_with_parens:
    print(f"  - {title}")

# Check for different quotation mark styles
titles_with_quotes = df_songs[df_songs['title'].str.contains(r'["""'']', regex=True, na=False)]['title'].head(20)
print("\nSample titles with quotation marks:")
for title in titles_with_quotes:
    print(f"  - {title}")


SONG TITLE ANALYSIS

Sample titles:
["It's All In The Game", "It's Only Make Believe", 'Little Star', 'Nel Blu Dipinto Di Blu (Volaré)', 'Poor Little Fool', 'Smoke Gets In Your Eyes', 'Stagger Lee', 'The Chipmunk Song', 'The Chipmunk Song', 'To Know Him, Is To Love Him', 'Tom Dooley', '16 Candles', 'Bird Dog', 'Donna', 'Jingle Bell Rock', 'My Happiness', 'Patricia', 'Patricia', 'Problems', 'Rock-in Robin']

Total songs: 38,383
Unique titles: 26,780

Potential issues found:
  Leading/trailing spaces: 0 (0.0%)
  Multiple spaces: 5 (0.0%)
  Parentheses (features/versions): 2,542 (6.6%)
  Brackets: 13 (0.0%)
  Quotation marks: 319 (0.8%)
  All caps: 377 (1.0%)
  Special chars (non-ASCII): 16 (0.0%)

Sample titles with parentheses:
  - Nel Blu Dipinto Di Blu (Volaré)
  - The Hawaiian Wedding Song (Ke Kali Nei Au)
  - Left Right Out Of Your Heart (Hi Lee Hi Lo Hi Lup Up Up)
  - (All of a Sudden) My Heart Sings
  - Volare (Nel Blu Dipinto Di Blu)
  - She Was Only Seventeen (He Was One Year Mo

In [17]:
def normalize_title(title):
    """Normalize title for API matching"""
    if pd.isna(title):
        return ""
    
    title = str(title)
    
    # Convert to title case if all caps
    if title.isupper():
        title = title.title()
    
    # Remove leading/trailing spaces
    title = title.strip()
    
    # Normalize internal spaces
    title = ' '.join(title.split())
    
    return title

# Apply to SONGS
df_songs['title_normalized'] = df_songs['title'].apply(normalize_title)

print("="*60)
print("SONGS - Title Normalization")
print("="*60)
print(f"✅ Created title_normalized column for songs")
print(f"Total songs: {len(df_songs):,}")

# Check all caps conversions
caps_songs = df_songs[df_songs['title'].str.isupper()][['title', 'title_normalized']].head(10)
if len(caps_songs) > 0:
    print(f"\nAll caps conversions (showing {len(caps_songs)}):")
    print(caps_songs.to_string(index=False))

# Apply to ALBUMS
df_albums['title_normalized'] = df_albums['title'].apply(normalize_title)

print("\n" + "="*60)
print("ALBUMS - Title Normalization")
print("="*60)
print(f"✅ Created title_normalized column for albums")
print(f"Total albums: {len(df_albums):,}")

# Check all caps conversions
caps_albums = df_albums[df_albums['title'].str.isupper()][['title', 'title_normalized']].head(10)
if len(caps_albums) > 0:
    print(f"\nAll caps conversions (showing {len(caps_albums)}):")
    print(caps_albums.to_string(index=False))

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print("Both dataframes now have title_normalized columns ready for API matching!")
print("Original 'title' columns preserved for reference.")


SONGS - Title Normalization
✅ Created title_normalized column for songs
Total songs: 38,383

All caps conversions (showing 10):
         title title_normalized
        M.T.A.           M.T.A.
      P.T. 109         P.T. 109
        8 X 10           8 X 10
        G.T.O.           G.T.O.
       S-W-I-M          S-W-I-M
       L-O-V-E          L-O-V-E
   L-O-N-E-L-Y      L-O-N-E-L-Y
N-E-R-V-O-U-S!   N-E-R-V-O-U-S!
       B-A-B-Y          B-A-B-Y
 D-I-V-O-R-C-E    D-I-V-O-R-C-E

ALBUMS - Title Normalization
✅ Created title_normalized column for albums
Total albums: 39,129

All caps conversions (showing 10):
        title title_normalized
          SRO              Sro
          TCB              Tcb
          TCB              Tcb
          TCB              Tcb
   200 M.P.H.       200 M.P.H.
D-I-V-O-R-C-E    D-I-V-O-R-C-E
          SRC              Src
        SSSSH            Ssssh
       M.P.G.           M.P.G.
         NRBQ             Nrbq

SUMMARY
Both dataframes now have title_normali

In [2]:
!pip install spotipy


In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Replace these with your actual credentials
CLIENT_ID = 'cd5e1bcfa10a4443994e9235abfd94f9'
CLIENT_SECRET = 'd0f8b48548b04d4480214bb7367c92ae'

# Set up authentication
auth_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

sp = spotipy.Spotify(auth_manager=auth_manager)

print("✅ Spotify API connected!")

# Test the connection
try:
    # Search for a test track
    results = sp.search(q='Bohemian Rhapsody Queen', type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        print(f"\n🎵 Test successful! Found: '{track['name']}' by {track['artists'][0]['name']}")
        print(f"   Popularity: {track['popularity']}")
        print(f"   Duration: {track['duration_ms']/1000:.0f} seconds")
    else:
        print("⚠️ No results found")
except Exception as e:
    print(f"❌ Error: {e}")


✅ Spotify API connected!

🎵 Test successful! Found: 'Bohemian Rhapsody - Remastered 2011' by Queen
   Popularity: 76
   Duration: 354 seconds


In [5]:
import pandas as pd

# Load the final versions (with split collaborations)
df_songs = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_final.csv')
df_albums = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_final.csv')

print("✅ Loaded final dataframes")
print(f"\ndf_songs: {df_songs.shape[0]:,} rows, {df_songs.shape[1]} columns")
print(f"df_albums: {df_albums.shape[0]:,} rows, {df_albums.shape[1]} columns")
print(f"\nColumns in df_songs: {list(df_songs.columns)}")


✅ Loaded final dataframes

df_songs: 38,383 rows, 8 columns
df_albums: 39,129 rows, 8 columns

Columns in df_songs: ['title', 'performer', 'peak_pos', 'wks_on_chart', 'first_charting_year', 'original_performer', 'performer_normalized', 'original_performer_normalized']


In [8]:
# Test with just basic track info (no audio features)
sample_songs = df_songs.head(5)

print("Testing Spotify data retrieval (basic info only)...\n")

for idx, row in sample_songs.iterrows():
    title = row['title']
    artist = row['performer_normalized']
    
    # Search for the track
    query = f"{title} {artist}"
    try:
        results = sp.search(q=query, type='track', limit=1)
        
        if results['tracks']['items']:
            track = results['tracks']['items'][0]
            
            print(f"🎵 {title} - {artist}")
            print(f"   Spotify Match: {track['name']} by {track['artists'][0]['name']}")
            print(f"   Popularity: {track['popularity']}")
            print(f"   Duration: {track['duration_ms']/1000:.0f} seconds")
            print(f"   Release Date: {track['album']['release_date']}")
            print(f"   Track ID: {track['id']}")
            print()
        else:
            print(f"❌ Not found: {title} - {artist}\n")
    except Exception as e:
        print(f"❌ Error for {title} - {artist}: {e}\n")


Testing Spotify data retrieval (basic info only)...

🎵 It's All In The Game - tommy edwards
   Spotify Match: It's All In The Game by Tommy Edwards
❌ Error for It's All In The Game - tommy edwards: 'popularity'

🎵 It's Only Make Believe - conway twitty
   Spotify Match: It's Only Make Believe by Conway Twitty
❌ Error for It's Only Make Believe - conway twitty: 'popularity'

🎵 Little Star - the elegants
   Spotify Match: Little Star by The Elegants
❌ Error for Little Star - the elegants: 'popularity'

🎵 Nel Blu Dipinto Di Blu (Volaré) - domenico modugno
   Spotify Match: Nel blu dipinto di blu (Volare) by Domenico Modugno
❌ Error for Nel Blu Dipinto Di Blu (Volaré) - domenico modugno: 'popularity'

🎵 Poor Little Fool - ricky nelson
   Spotify Match: Poor Little Fool by Ricky Nelson
❌ Error for Poor Little Fool - ricky nelson: 'popularity'



In [9]:
# Check what data Spotify actually returns
sample_songs = df_songs.head(3)

print("Checking available Spotify data...\n")

for idx, row in sample_songs.iterrows():
    title = row['title']
    artist = row['performer_normalized']
    
    query = f"{title} {artist}"
    try:
        results = sp.search(q=query, type='track', limit=1)
        
        if results['tracks']['items']:
            track = results['tracks']['items'][0]
            
            print(f"🎵 {title} - {artist}")
            print(f"   Spotify Match: {track['name']} by {track['artists'][0]['name']}")
            print(f"\n   Available fields:")
            print(f"   - Track ID: {track.get('id', 'N/A')}")
            print(f"   - Popularity: {track.get('popularity', 'N/A')}")
            print(f"   - Duration: {track.get('duration_ms', 'N/A')}")
            print(f"   - Explicit: {track.get('explicit', 'N/A')}")
            print(f"   - Preview URL: {track.get('preview_url', 'N/A')}")
            
            # Check album info
            if 'album' in track:
                print(f"   - Album: {track['album'].get('name', 'N/A')}")
                print(f"   - Release Date: {track['album'].get('release_date', 'N/A')}")
            
            # Print ALL available keys to see what we have
            print(f"\n   All track keys: {list(track.keys())}")
            print("="*60 + "\n")
        else:
            print(f"❌ Not found: {title} - {artist}\n")
    except Exception as e:
        print(f"❌ Error for {title} - {artist}: {e}\n")


Checking available Spotify data...

🎵 It's All In The Game - tommy edwards
   Spotify Match: It's All In The Game by Tommy Edwards

   Available fields:
   - Track ID: 2tvt5K7y1gndmCgtIoLo1f
   - Popularity: N/A
   - Duration: 158520
   - Explicit: False
   - Preview URL: N/A
   - Album: It’s All In The Game
   - Release Date: 1958-07-08

   All track keys: ['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'track_number', 'type', 'uri']

🎵 It's Only Make Believe - conway twitty
   Spotify Match: It's Only Make Believe by Conway Twitty

   Available fields:
   - Track ID: 1xVOttVNT27FBTD8iHjOfU
   - Popularity: N/A
   - Duration: 132026
   - Explicit: False
   - Preview URL: N/A
   - Album: Conway Twitty Sings
   - Release Date: 1959-01-01

   All track keys: ['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'track_number', 'type',

In [10]:
# Test audio features with the track IDs we just found
test_track_ids = [
    '2tvt5K7y1gndmCgtIoLo1f',  # It's All In The Game
    '1xVOttVNT27FBTD8iHjOfU',  # It's Only Make Believe
    '6xupOaBWORbDmakCdQwMRG'   # Little Star
]

print("Testing audio features access...\n")

try:
    audio_features = sp.audio_features(test_track_ids)
    
    for i, features in enumerate(audio_features):
        print(f"Track {i+1}:")
        if features:
            print(f"  ✅ Audio features available!")
            print(f"     Danceability: {features.get('danceability', 'N/A')}")
            print(f"     Energy: {features.get('energy', 'N/A')}")
            print(f"     Valence: {features.get('valence', 'N/A')}")
            print(f"     Tempo: {features.get('tempo', 'N/A')}")
            print(f"     All keys: {list(features.keys())}")
        else:
            print(f"  ❌ No audio features available")
        print()
        
except Exception as e:
    print(f"❌ Error accessing audio features: {e}")
    print(f"   Error type: {type(e).__name__}")


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2tvt5K7y1gndmCgtIoLo1f,1xVOttVNT27FBTD8iHjOfU,6xupOaBWORbDmakCdQwMRG with Params: {} returned 403 due to None


Testing audio features access...

❌ Error accessing audio features: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=2tvt5K7y1gndmCgtIoLo1f,1xVOttVNT27FBTD8iHjOfU,6xupOaBWORbDmakCdQwMRG:
 None, reason: None
   Error type: SpotifyException


In [12]:
import requests
from urllib.parse import quote
import time

def get_musicbrainz_tags(title, artist, max_retries=2):
    """
    Get genre tags from MusicBrainz for a recording
    Returns: (tags_list, recording_mbid)
    """
    # Search for the recording
    query = f'recording:"{title}" AND artist:"{artist}"'
    encoded_query = quote(query)
    search_url = f"https://musicbrainz.org/ws/2/recording/?query={encoded_query}&fmt=json&limit=1"
    
    headers = {'User-Agent': 'BillboardDataAnalysis/1.0 (educational-project)'}
    
    for attempt in range(max_retries):
        try:
            # Search for recording
            response = requests.get(search_url, headers=headers, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                
                if data.get('recordings') and len(data['recordings']) > 0:
                    recording = data['recordings'][0]
                    mbid = recording['id']
                    
                    # Now get the full recording with tags
                    time.sleep(1.1)  # Rate limiting
                    tags_url = f"https://musicbrainz.org/ws/2/recording/{mbid}?inc=tags&fmt=json"
                    tags_response = requests.get(tags_url, headers=headers, timeout=10)
                    
                    if tags_response.status_code == 200:
                        recording_data = tags_response.json()
                        tags = recording_data.get('tags', [])
                        
                        # Extract tag names and counts
                        tag_list = [
                            {'name': tag['name'], 'count': tag.get('count', 0)} 
                            for tag in tags
                        ]
                        
                        return tag_list, mbid
                    
                return None, None
            
            elif response.status_code == 503:
                time.sleep(2)
                continue
            else:
                return None, None
                
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            return None, None
    
    return None, None

# Test with sample songs
print("Testing MusicBrainz tag retrieval...\n")

sample_songs = df_songs.head(25)

for idx, row in sample_songs.iterrows():
    title = row['title']
    artist = row['performer_normalized']
    
    print(f"🎵 {title} - {artist}")
    tags, mbid = get_musicbrainz_tags(title, artist)
    
    if tags:
        print(f"   MBID: {mbid}")
        print(f"   Tags found: {len(tags)}")
        # Show top tags
        sorted_tags = sorted(tags, key=lambda x: x['count'], reverse=True)[:5]
        for tag in sorted_tags:
            print(f"      - {tag['name']} (count: {tag['count']})")
    else:
        print(f"   ❌ No tags found")
    
    print()
    time.sleep(1.1)  # Rate limiting


Testing MusicBrainz tag retrieval...

🎵 It's All In The Game - tommy edwards
   ❌ No tags found

🎵 It's Only Make Believe - conway twitty
   MBID: 6640900a-0b17-4f12-81e9-a67fab07447b
   Tags found: 2
      - classic country (count: 1)
      - country (count: 1)

🎵 Little Star - the elegants
   ❌ No tags found

🎵 Nel Blu Dipinto Di Blu (Volaré) - domenico modugno
   ❌ No tags found

🎵 Poor Little Fool - ricky nelson
   ❌ No tags found

🎵 Smoke Gets In Your Eyes - the platters
   ❌ No tags found

🎵 Stagger Lee - lloyd price
   ❌ No tags found

🎵 The Chipmunk Song - the chipmunks
   ❌ No tags found

🎵 The Chipmunk Song - david seville
   ❌ No tags found

🎵 To Know Him, Is To Love Him - the teddy bears
   MBID: 20b42183-40be-4e9d-bb19-ef7ea52044ae
   Tags found: 2
      - rock roll (count: 3)
      - country (count: 1)

🎵 Tom Dooley - the kingston trio
   ❌ No tags found

🎵 16 Candles - the crests
   MBID: 1cfcace3-c908-4cbe-8914-3eebf92b4c65
   Tags found: 1
      - doo-wwop (count: 1)



In [13]:
import requests
from urllib.parse import quote
import time

def get_artist_tags_from_mb(artist_name, max_retries=2):
    """
    Get genre tags for an artist from MusicBrainz
    Returns: (tags_list, artist_mbid)
    """
    query = f'artist:"{artist_name}"'
    encoded_query = quote(query)
    search_url = f"https://musicbrainz.org/ws/2/artist/?query={encoded_query}&fmt=json&limit=1"
    
    headers = {'User-Agent': 'BillboardDataAnalysis/1.0 (educational-project)'}
    
    for attempt in range(max_retries):
        try:
            # Search for artist
            response = requests.get(search_url, headers=headers, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                
                if data.get('artists') and len(data['artists']) > 0:
                    artist = data['artists'][0]
                    mbid = artist['id']
                    
                    # Get full artist data with tags
                    time.sleep(1.1)  # Rate limiting
                    tags_url = f"https://musicbrainz.org/ws/2/artist/{mbid}?inc=tags&fmt=json"
                    tags_response = requests.get(tags_url, headers=headers, timeout=10)
                    
                    if tags_response.status_code == 200:
                        artist_data = tags_response.json()
                        tags = artist_data.get('tags', [])
                        
                        tag_list = [
                            {'name': tag['name'], 'count': tag.get('count', 0)} 
                            for tag in tags
                        ]
                        
                        return tag_list, mbid
                    
                return None, None
            
            elif response.status_code == 503:
                time.sleep(2)
                continue
            else:
                return None, None
                
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            return None, None
    
    return None, None

# Test with a sample of unique artists first
unique_artists = df_songs['performer_normalized'].unique()
print(f"Total unique artists in df_songs: {len(unique_artists):,}\n")

# Sample first 20 artists
sample_artists = unique_artists[:20]

print("Testing artist tag retrieval (first 20 artists)...\n")

artists_with_tags = 0
artists_without_tags = 0

for artist in sample_artists:
    tags, mbid = get_artist_tags_from_mb(artist)
    
    if tags and len(tags) > 0:
        artists_with_tags += 1
        sorted_tags = sorted(tags, key=lambda x: x['count'], reverse=True)[:3]
        tag_names = [t['name'] for t in sorted_tags]
        print(f"✅ {artist}: {', '.join(tag_names)}")
    else:
        artists_without_tags += 1
        print(f"❌ {artist}: No tags")

print(f"\n{'='*60}")
print(f"SAMPLE RESULTS (first 20 artists):")
print(f"{'='*60}")
print(f"With tags: {artists_with_tags} ({artists_with_tags/20*100:.1f}%)")
print(f"Without tags: {artists_without_tags} ({artists_without_tags/20*100:.1f}%)")
print(f"\nEstimated artists with tags (all {len(unique_artists):,}): ~{int(len(unique_artists) * artists_with_tags/20):,}")


Total unique artists in df_songs: 8,903

Testing artist tag retrieval (first 20 artists)...

❌ tommy edwards: No tags
✅ conway twitty: country, 2008 universal fire victim, nashville sound
✅ the elegants: doo-wop
✅ domenico modugno: pop, singer-songwriter, traditional pop
❌ ricky nelson: No tags
✅ the platters: doo-wop, traditional pop, pop
✅ lloyd price: new orleans r&b, 2008 universal fire victim, pop soul
✅ the chipmunks: fictitious artist, aln-sh, bogus artist
✅ david seville: children's music, christmas music, fictitious artist
❌ the teddy bears: No tags
✅ the kingston trio: folk, american folk music, american folk revival
✅ the crests: doo-wop, r&b
✅ the everly brothers: pop, rock & roll, rockabilly
✅ ritchie valens: chicano rock, death by plane crash, rock & roll
✅ bobby helms: 2008 universal fire victim, christmas music, country
✅ connie francis: pop, american, brill building
✅ perez prado: cuban, latin, mambo
✅ his orchestra: big band, jazz, swing
✅ bobby day: soul
❌ bill parso

In [14]:
# Create combined dataframe with ALL songs and albums (stacked)
df_songs_all = df_songs.copy()
df_albums_all = df_albums.copy()

# Add record type column to distinguish songs from albums
df_songs_all['record_type'] = 'song'
df_albums_all['record_type'] = 'album'

# Stack them together
df_combined = pd.concat([df_songs_all, df_albums_all], ignore_index=True)

print("="*60)
print("COMBINED DATAFRAME (EVERYONE)")
print("="*60)
print(f"Total rows: {len(df_combined):,}")
print(f"  Songs: {len(df_songs_all):,}")
print(f"  Albums: {len(df_albums_all):,}")
print(f"Total unique performers: {df_combined['performer_normalized'].nunique():,}")
print("="*60)

# Breakdown by performer type
performers_both = set(df_songs_all['performer_normalized'].unique()) & \
                 set(df_albums_all['performer_normalized'].unique())
performers_songs_only = set(df_songs_all['performer_normalized'].unique()) - \
                        set(df_albums_all['performer_normalized'].unique())
performers_albums_only = set(df_albums_all['performer_normalized'].unique()) - \
                         set(df_songs_all['performer_normalized'].unique())

print(f"\nPerformer breakdown:")
print(f"  In BOTH songs & albums: {len(performers_both):,}")
print(f"  Songs ONLY: {len(performers_songs_only):,}")
print(f"  Albums ONLY: {len(performers_albums_only):,}")

# Show sample
print(f"\nSample rows:")
print(df_combined[['performer', 'title', 'record_type', 'peak_pos']].head(10))


COMBINED DATAFRAME (EVERYONE)
Total rows: 77,512
  Songs: 38,383
  Albums: 39,129
Total unique performers: 14,226

Performer breakdown:
  In BOTH songs & albums: 4,654
  Songs ONLY: 4,249
  Albums ONLY: 5,323

Sample rows:
          performer                            title record_type  peak_pos
0     Tommy Edwards             It's All In The Game        song         1
1     Conway Twitty           It's Only Make Believe        song         1
2      The Elegants                      Little Star        song         1
3  Domenico Modugno  Nel Blu Dipinto Di Blu (Volaré)        song         1
4      Ricky Nelson                 Poor Little Fool        song         1
5      The Platters          Smoke Gets In Your Eyes        song         1
6       Lloyd Price                      Stagger Lee        song         1
7     The Chipmunks                The Chipmunk Song        song         1
8     David Seville                The Chipmunk Song        song         1
9   The Teddy Bears      To

In [15]:
# Save combined dataframe to CSV
output_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_songs_and_albums_combined_all_take2.csv'

df_combined.to_csv(output_path, index=False)

print("✅ Saved combined dataframe to CSV")
print(f"📁 File: {output_path}")
print(f"📊 Total rows: {len(df_combined):,}")
print(f"   Songs: {len(df_combined[df_combined['record_type'] == 'song']):,}")
print(f"   Albums: {len(df_combined[df_combined['record_type'] == 'album']):,}")
print(f"🎤 Unique performers: {df_combined['performer_normalized'].nunique():,}")
print(f"📋 Columns: {len(df_combined.columns)}")


✅ Saved combined dataframe to CSV
📁 File: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_songs_and_albums_combined_all_take2.csv
📊 Total rows: 77,512
   Songs: 38,383
   Albums: 39,129
🎤 Unique performers: 14,226
📋 Columns: 9


In [16]:
# Sort by performer_normalized
df_combined_sorted = df_combined.sort_values('performer_normalized').reset_index(drop=True)

print("✅ Sorted by artist (performer_normalized)")
print(f"\nSample (first 20 rows):")
print(df_combined_sorted[['performer_normalized', 'title', 'record_type', 'peak_pos', 'first_charting_year']].head(20))

✅ Sorted by artist (performer_normalized)

Sample (first 20 rows):
      performer_normalized                                      title  \
0        !!! (chk chk chk)                                 Myth Takes   
1          "groove" holmes                                      Misty   
2          "groove" holmes                           What Now My Love   
3   "little" jimmy dickens  May The Bird Of Paradise Fly Up Your Nose   
4          "pookie" hudson                              I Know I Know   
5      "weird al" yankovic                                     Eat It   
6      "weird al" yankovic                              White & Nerdy   
7      "weird al" yankovic                                      Ricky   
8      "weird al" yankovic                         I Lost On Jeopardy   
9      "weird al" yankovic                                Word Crimes   
10     "weird al" yankovic                        Smells Like Nirvana   
11     "weird al" yankovic                             Ca

In [17]:
# Create simple artist-level dataframe (one row per artist)
df_artists = df_combined[['performer_normalized', 'performer']].drop_duplicates('performer_normalized').reset_index(drop=True)

# Sort alphabetically
df_artists = df_artists.sort_values('performer_normalized').reset_index(drop=True)

print(f"✅ Created df_artists")
print(f"Total unique artists: {len(df_artists):,}")
print(f"\nFirst 20 artists:")
print(df_artists.head(20))


✅ Created df_artists
Total unique artists: 14,226

First 20 artists:
            performer_normalized                     performer
0              !!! (chk chk chk)             !!! (Chk Chk Chk)
1                "groove" holmes               "Groove" Holmes
2         "little" jimmy dickens        "Little" Jimmy Dickens
3                "pookie" hudson               "Pookie" Hudson
4            "weird al" yankovic           "Weird Al" Yankovic
5                           $not                          $NOT
6                    $uicideboy$                   $uicideboy$
7             $uicideboy$ x germ            $uicideBoy$ x Germ
8               & cardi b or nas              & Cardi B Or Nas
9                  & jack harlow                 & Jack Harlow
10                 & johnny cash                 & Johnny Cash
11                    & mystikal                    & Mystikal
12                     & sam dew                     & Sam Dew
13  & youngboy never broke again  & Youngboy Neve

In [18]:
print("Columns in df_songs:")
print(df_songs.columns.tolist())
print("\nColumns in df_albums:")
print(df_albums.columns.tolist())

Columns in df_songs:
['title', 'performer', 'peak_pos', 'wks_on_chart', 'first_charting_year', 'original_performer', 'performer_normalized', 'original_performer_normalized']

Columns in df_albums:
['title', 'performer', 'peak_pos', 'wks_on_chart', 'first_charting_year', 'original_performer', 'performer_normalized', 'original_performer_normalized']


In [19]:
import pandas as pd
import numpy as np

# Start with unique performers
df_artists = df_songs[['performer_normalized', 'performer']].drop_duplicates('performer_normalized').reset_index(drop=True)

# Get unique performers from albums too (in case some only have albums)
df_artists_albums = df_albums[['performer_normalized', 'performer']].drop_duplicates('performer_normalized')
df_artists = pd.concat([df_artists, df_artists_albums]).drop_duplicates('performer_normalized').reset_index(drop=True)

# === SONG STATISTICS ===

# Basic song aggregations
songs_agg = df_songs.groupby('performer_normalized').agg({
    'first_charting_year': ['min', 'max'],
    'title': 'count',
    'wks_on_chart': 'sum',
    'peak_pos': 'min'
}).reset_index()
songs_agg.columns = ['performer_normalized', 'first_song_year', 'last_song_year', 
                     'total_charting_songs', 'total_charting_songs_duration_weeks', 
                     'highest_charting_song_position']

# Top hits counts for songs
top_1_songs = df_songs[df_songs['peak_pos'] == 1].groupby('performer_normalized').size().reset_index(name='#1_hit_song_count')
top_10_songs = df_songs[df_songs['peak_pos'] <= 10].groupby('performer_normalized').size().reset_index(name='top_10_song_count')
top_20_songs = df_songs[df_songs['peak_pos'] <= 20].groupby('performer_normalized').size().reset_index(name='top_20_song_count')
top_50_songs = df_songs[df_songs['peak_pos'] <= 50].groupby('performer_normalized').size().reset_index(name='top_50_song_count')

# Get highest charting song name (song with best peak position)
highest_songs = df_songs.loc[df_songs.groupby('performer_normalized')['peak_pos'].idxmin()][['performer_normalized', 'title']].rename(columns={'title': 'highest_charting_song_name'})

# Get first charting song details (earliest year)
first_songs = df_songs.loc[df_songs.groupby('performer_normalized')['first_charting_year'].idxmin()][['performer_normalized', 'title', 'peak_pos', 'wks_on_chart']]
first_songs = first_songs.rename(columns={
    'title': 'first_charting_song_name',
    'peak_pos': 'first_charting_song_position',
    'wks_on_chart': 'first_charting_song_duration'
})

# === ALBUM STATISTICS ===

# Basic album aggregations
albums_agg = df_albums.groupby('performer_normalized').agg({
    'first_charting_year': ['min', 'max'],
    'title': 'count',
    'wks_on_chart': 'sum',
    'peak_pos': 'min'
}).reset_index()
albums_agg.columns = ['performer_normalized', 'first_album_year', 'last_album_year',
                      'total_charting_albums', 'total_charting_albums_duration_weeks',
                      'highest_charting_album_position']

# Top hits counts for albums
top_1_albums = df_albums[df_albums['peak_pos'] == 1].groupby('performer_normalized').size().reset_index(name='#1_hit_album_count')
top_10_albums = df_albums[df_albums['peak_pos'] <= 10].groupby('performer_normalized').size().reset_index(name='top_10_album_count')
top_20_albums = df_albums[df_albums['peak_pos'] <= 20].groupby('performer_normalized').size().reset_index(name='top_20_album_count')
top_50_albums = df_albums[df_albums['peak_pos'] <= 50].groupby('performer_normalized').size().reset_index(name='top_50_album_count')

# Get highest charting album name
highest_albums = df_albums.loc[df_albums.groupby('performer_normalized')['peak_pos'].idxmin()][['performer_normalized', 'title']].rename(columns={'title': 'highest_charting_album_name'})

# Get first charting album details
first_albums = df_albums.loc[df_albums.groupby('performer_normalized')['first_charting_year'].idxmin()][['performer_normalized', 'title', 'peak_pos', 'wks_on_chart']]
first_albums = first_albums.rename(columns={
    'title': 'first_charting_album_name',
    'peak_pos': 'first_charting_album_position',
    'wks_on_chart': 'first_charting_album_duration'
})

# === YEARS ACTIVE CALCULATIONS ===

# Combine both dataframes to calculate overall years active
df_both = pd.concat([
    df_songs[['performer_normalized', 'first_charting_year']],
    df_albums[['performer_normalized', 'first_charting_year']]
])

years_active = df_both.groupby('performer_normalized').agg({
    'first_charting_year': ['min', 'max']
}).reset_index()
years_active.columns = ['performer_normalized', 'overall_first_year', 'overall_last_year']

# Create years_active_on_charts string
years_active['years_active_on_charts'] = years_active['overall_first_year'].astype(int).astype(str) + '-' + years_active['overall_last_year'].astype(int).astype(str)

# Calculate number of unique years active
unique_years = df_both.groupby('performer_normalized')['first_charting_year'].nunique().reset_index(name='#_of_years_active')

# === MERGE EVERYTHING ===

df_artists = df_artists.merge(songs_agg, on='performer_normalized', how='left')
df_artists = df_artists.merge(albums_agg, on='performer_normalized', how='left')
df_artists = df_artists.merge(years_active[['performer_normalized', 'years_active_on_charts']], on='performer_normalized', how='left')
df_artists = df_artists.merge(unique_years, on='performer_normalized', how='left')

df_artists = df_artists.merge(top_1_songs, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_10_songs, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_20_songs, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_50_songs, on='performer_normalized', how='left')

df_artists = df_artists.merge(top_1_albums, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_10_albums, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_20_albums, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_50_albums, on='performer_normalized', how='left')

df_artists = df_artists.merge(highest_songs, on='performer_normalized', how='left')
df_artists = df_artists.merge(first_songs, on='performer_normalized', how='left')

df_artists = df_artists.merge(highest_albums, on='performer_normalized', how='left')
df_artists = df_artists.merge(first_albums, on='performer_normalized', how='left')

# Fill NaN values with 0 for count columns, keep NaN for text columns
count_columns = ['first_song_year', 'last_song_year', 'total_charting_songs', 
                 'total_charting_songs_duration_weeks', 'highest_charting_song_position',
                 'first_album_year', 'last_album_year', 'total_charting_albums',
                 'total_charting_albums_duration_weeks', 'highest_charting_album_position',
                 '#_of_years_active', '#1_hit_song_count', 'top_10_song_count',
                 'top_20_song_count', 'top_50_song_count', '#1_hit_album_count',
                 'top_10_album_count', 'top_20_album_count', 'top_50_album_count',
                 'first_charting_song_position', 'first_charting_song_duration',
                 'first_charting_album_position', 'first_charting_album_duration']

df_artists[count_columns] = df_artists[count_columns].fillna(0)

# Reorder columns to match your requested order
df_artists = df_artists[[
    'performer_normalized',
    'performer',
    'first_song_year',
    'last_song_year',
    'first_album_year',
    'last_album_year',
    'years_active_on_charts',
    '#_of_years_active',
    'total_charting_songs',
    'total_charting_albums',
    '#1_hit_song_count',
    '#1_hit_album_count',
    'top_10_song_count',
    'top_10_album_count',
    'top_20_song_count',
    'top_20_album_count',
    'top_50_song_count',
    'top_50_album_count',
    'highest_charting_song_name',
    'highest_charting_song_position',
    'first_charting_song_name',
    'first_charting_song_position',
    'first_charting_song_duration',
    'total_charting_songs_duration_weeks',
    'highest_charting_album_name',
    'highest_charting_album_position',
    'first_charting_album_name',
    'first_charting_album_position',
    'first_charting_album_duration',
    'total_charting_albums_duration_weeks'
]]

# Sort by performer_normalized
df_artists = df_artists.sort_values('performer_normalized').reset_index(drop=True)

print(f"Created df_artists with {len(df_artists)} artists and {len(df_artists.columns)} columns")
df_artists.head(10)


Created df_artists with 14226 artists and 30 columns


,performer_normalized,performer,first_song_year,last_song_year,first_album_year,last_album_year,years_active_on_charts,#_of_years_active,total_charting_songs,total_charting_albums,...,first_charting_song_name,first_charting_song_position,first_charting_song_duration,total_charting_songs_duration_weeks,highest_charting_album_name,highest_charting_album_position,first_charting_album_name,first_charting_album_position,first_charting_album_duration,total_charting_albums_duration_weeks
0,!!! (chk chk chk),!!! (Chk Chk Chk),0.0,0.0,2007.0,2007.0,2007-2007,1,0.0,1.0,...,NaN,0.0,0.0,0.0,Myth Takes,195.0,Myth Takes,195.0,1.0,1.0
1,"""groove"" holmes","""Groove"" Holmes",1966.0,1966.0,0.0,0.0,1966-1966,1,2.0,0.0,...,Misty,44.0,11.0,14.0,NaN,0.0,NaN,0.0,0.0,0.0
2,"""little"" jimmy dickens","""Little"" Jimmy Dickens",1965.0,1965.0,0.0,0.0,1965-1965,1,1.0,0.0,...,May The Bird Of Paradise Fly Up Your Nose,15.0,10.0,10.0,NaN,0.0,NaN,0.0,0.0,0.0
3,"""pookie"" hudson","""Pookie"" Hudson",1963.0,1963.0,0.0,0.0,1963-1963,1,1.0,0.0,...,I Know I Know,96.0,1.0,1.0,NaN,0.0,NaN,0.0,0.0,0.0
4,"""weird al"" yankovic","""Weird Al"" Yankovic",1983.0,2014.0,0.0,0.0,1983-2014,8,11.0,0.0,...,Ricky,63.0,8.0,91.0,NaN,0.0,NaN,0.0,0.0,0.0
5,$not,$NOT,2021.0,2022.0,2020.0,2022.0,2020-2022,3,2.0,2.0,...,Tell Em,64.0,4.0,5.0,Ethereal,66.0,Beautiful Havoc,172.0,1.0,2.0
6,$uicideboy$,$uicideboy$,2024.0,2026.0,2018.0,2026.0,2018-2026,8,9.0,8.0,...,The Thin Grey Line,71.0,1.0,11.0,Thy Kingdom Come,4.0,I Want To Die In New Orleans,9.0,4.0,91.0
7,$uicideboy$ x germ,$uicideBoy$ x Germ,0.0,0.0,2022.0,2022.0,2022-2022,1,0.0,1.0,...,NaN,0.0,0.0,0.0,DirtiestNastiest$uicide,54.0,DirtiestNastiest$uicide,54.0,2.0,2.0
8,& cardi b or nas,& Cardi B Or Nas,2019.0,2019.0,0.0,0.0,2019-2019,1,1.0,0.0,...,Rodeo,22.0,9.0,9.0,NaN,0.0,NaN,0.0,0.0,0.0
9,& jack harlow,& Jack Harlow,2021.0,2021.0,0.0,0.0,2021-2021,1,1.0,0.0,...,Industry Baby,1.0,42.0,42.0,NaN,0.0,NaN,0.0,0.0,0.0


In [20]:
import pandas as pd
import numpy as np

# Start with unique performers
df_artists = df_songs[['performer_normalized', 'performer']].drop_duplicates('performer_normalized').reset_index(drop=True)

# Get unique performers from albums too (in case some only have albums)
df_artists_albums = df_albums[['performer_normalized', 'performer']].drop_duplicates('performer_normalized')
df_artists = pd.concat([df_artists, df_artists_albums]).drop_duplicates('performer_normalized').reset_index(drop=True)

# === SONG STATISTICS ===

# Basic song aggregations
songs_agg = df_songs.groupby('performer_normalized').agg({
    'first_charting_year': ['min', 'max'],
    'title': 'count',
    'wks_on_chart': 'sum',
    'peak_pos': 'min'
}).reset_index()
songs_agg.columns = ['performer_normalized', 'first_song_year', 'last_song_year', 
                     'total_charting_songs', 'total_charting_songs_duration_weeks', 
                     'highest_charting_song_position']

# Top hits counts for songs
top_1_songs = df_songs[df_songs['peak_pos'] == 1].groupby('performer_normalized').size().reset_index(name='#1_hit_song_count')
top_10_songs = df_songs[df_songs['peak_pos'] <= 10].groupby('performer_normalized').size().reset_index(name='top_10_song_count')
top_20_songs = df_songs[df_songs['peak_pos'] <= 20].groupby('performer_normalized').size().reset_index(name='top_20_song_count')
top_50_songs = df_songs[df_songs['peak_pos'] <= 50].groupby('performer_normalized').size().reset_index(name='top_50_song_count')

# Get highest charting song name (song with best peak position)
highest_songs = df_songs.loc[df_songs.groupby('performer_normalized')['peak_pos'].idxmin()][['performer_normalized', 'title']].rename(columns={'title': 'highest_charting_song_name'})

# Get first charting song details (earliest year)
first_songs = df_songs.loc[df_songs.groupby('performer_normalized')['first_charting_year'].idxmin()][['performer_normalized', 'title', 'peak_pos', 'wks_on_chart']]
first_songs = first_songs.rename(columns={
    'title': 'first_charting_song_name',
    'peak_pos': 'first_charting_song_position',
    'wks_on_chart': 'first_charting_song_duration'
})

# === ALBUM STATISTICS ===

# Basic album aggregations
albums_agg = df_albums.groupby('performer_normalized').agg({
    'first_charting_year': ['min', 'max'],
    'title': 'count',
    'wks_on_chart': 'sum',
    'peak_pos': 'min'
}).reset_index()
albums_agg.columns = ['performer_normalized', 'first_album_year', 'last_album_year',
                      'total_charting_albums', 'total_charting_albums_duration_weeks',
                      'highest_charting_album_position']

# Top hits counts for albums
top_1_albums = df_albums[df_albums['peak_pos'] == 1].groupby('performer_normalized').size().reset_index(name='#1_hit_album_count')
top_10_albums = df_albums[df_albums['peak_pos'] <= 10].groupby('performer_normalized').size().reset_index(name='top_10_album_count')
top_20_albums = df_albums[df_albums['peak_pos'] <= 20].groupby('performer_normalized').size().reset_index(name='top_20_album_count')
top_50_albums = df_albums[df_albums['peak_pos'] <= 50].groupby('performer_normalized').size().reset_index(name='top_50_album_count')

# Get highest charting album name
highest_albums = df_albums.loc[df_albums.groupby('performer_normalized')['peak_pos'].idxmin()][['performer_normalized', 'title']].rename(columns={'title': 'highest_charting_album_name'})

# Get first charting album details
first_albums = df_albums.loc[df_albums.groupby('performer_normalized')['first_charting_year'].idxmin()][['performer_normalized', 'title', 'peak_pos', 'wks_on_chart']]
first_albums = first_albums.rename(columns={
    'title': 'first_charting_album_name',
    'peak_pos': 'first_charting_album_position',
    'wks_on_chart': 'first_charting_album_duration'
})

# === YEARS ACTIVE CALCULATIONS ===

# Combine both dataframes to calculate overall years active
df_both = pd.concat([
    df_songs[['performer_normalized', 'first_charting_year']],
    df_albums[['performer_normalized', 'first_charting_year']]
])

years_active = df_both.groupby('performer_normalized').agg({
    'first_charting_year': ['min', 'max']
}).reset_index()
years_active.columns = ['performer_normalized', 'overall_first_year', 'overall_last_year']

# Create years_active_on_charts string
years_active['years_active_on_charts'] = years_active['overall_first_year'].astype(int).astype(str) + '-' + years_active['overall_last_year'].astype(int).astype(str)

# Calculate number of unique years active
unique_years = df_both.groupby('performer_normalized')['first_charting_year'].nunique().reset_index(name='#_of_unique_years_active')

# === MERGE EVERYTHING ===

df_artists = df_artists.merge(songs_agg, on='performer_normalized', how='left')
df_artists = df_artists.merge(albums_agg, on='performer_normalized', how='left')
df_artists = df_artists.merge(years_active[['performer_normalized', 'years_active_on_charts']], on='performer_normalized', how='left')
df_artists = df_artists.merge(unique_years, on='performer_normalized', how='left')

df_artists = df_artists.merge(top_1_songs, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_10_songs, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_20_songs, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_50_songs, on='performer_normalized', how='left')

df_artists = df_artists.merge(top_1_albums, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_10_albums, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_20_albums, on='performer_normalized', how='left')
df_artists = df_artists.merge(top_50_albums, on='performer_normalized', how='left')

df_artists = df_artists.merge(highest_songs, on='performer_normalized', how='left')
df_artists = df_artists.merge(first_songs, on='performer_normalized', how='left')

df_artists = df_artists.merge(highest_albums, on='performer_normalized', how='left')
df_artists = df_artists.merge(first_albums, on='performer_normalized', how='left')

# Fill NaN values with 0 for numeric columns, keep NaN for text columns
numeric_columns = ['first_song_year', 'last_song_year', 'total_charting_songs', 
                   'total_charting_songs_duration_weeks', 'highest_charting_song_position',
                   'first_album_year', 'last_album_year', 'total_charting_albums',
                   'total_charting_albums_duration_weeks', 'highest_charting_album_position',
                   '#_of_unique_years_active', '#1_hit_song_count', 'top_10_song_count',
                   'top_20_song_count', 'top_50_song_count', '#1_hit_album_count',
                   'top_10_album_count', 'top_20_album_count', 'top_50_album_count',
                   'first_charting_song_position', 'first_charting_song_duration',
                   'first_charting_album_position', 'first_charting_album_duration']

# Fill NaN with 0 and convert all to integers
df_artists[numeric_columns] = df_artists[numeric_columns].fillna(0).astype(int)

# Reorder columns to match your requested order
df_artists = df_artists[[
    'performer_normalized',
    'performer',
    'first_song_year',
    'last_song_year',
    'first_album_year',
    'last_album_year',
    'years_active_on_charts',
    '#_of_unique_years_active',
    'total_charting_songs',
    'total_charting_albums',
    '#1_hit_song_count',
    '#1_hit_album_count',
    'top_10_song_count',
    'top_10_album_count',
    'top_20_song_count',
    'top_20_album_count',
    'top_50_song_count',
    'top_50_album_count',
    'highest_charting_song_name',
    'highest_charting_song_position',
    'first_charting_song_name',
    'first_charting_song_position',
    'first_charting_song_duration',
    'total_charting_songs_duration_weeks',
    'highest_charting_album_name',
    'highest_charting_album_position',
    'first_charting_album_name',
    'first_charting_album_position',
    'first_charting_album_duration',
    'total_charting_albums_duration_weeks'
]]

# Sort by performer_normalized
df_artists = df_artists.sort_values('performer_normalized').reset_index(drop=True)

print(f"Created df_artists with {len(df_artists)} artists and {len(df_artists.columns)} columns")
df_artists.head(10)


Created df_artists with 14226 artists and 30 columns


,performer_normalized,performer,first_song_year,last_song_year,first_album_year,last_album_year,years_active_on_charts,#_of_unique_years_active,total_charting_songs,total_charting_albums,...,first_charting_song_name,first_charting_song_position,first_charting_song_duration,total_charting_songs_duration_weeks,highest_charting_album_name,highest_charting_album_position,first_charting_album_name,first_charting_album_position,first_charting_album_duration,total_charting_albums_duration_weeks
0,!!! (chk chk chk),!!! (Chk Chk Chk),0,0,2007,2007,2007-2007,1,0,1,...,NaN,0,0,0,Myth Takes,195,Myth Takes,195,1,1
1,"""groove"" holmes","""Groove"" Holmes",1966,1966,0,0,1966-1966,1,2,0,...,Misty,44,11,14,NaN,0,NaN,0,0,0
2,"""little"" jimmy dickens","""Little"" Jimmy Dickens",1965,1965,0,0,1965-1965,1,1,0,...,May The Bird Of Paradise Fly Up Your Nose,15,10,10,NaN,0,NaN,0,0,0
3,"""pookie"" hudson","""Pookie"" Hudson",1963,1963,0,0,1963-1963,1,1,0,...,I Know I Know,96,1,1,NaN,0,NaN,0,0,0
4,"""weird al"" yankovic","""Weird Al"" Yankovic",1983,2014,0,0,1983-2014,8,11,0,...,Ricky,63,8,91,NaN,0,NaN,0,0,0
5,$not,$NOT,2021,2022,2020,2022,2020-2022,3,2,2,...,Tell Em,64,4,5,Ethereal,66,Beautiful Havoc,172,1,2
6,$uicideboy$,$uicideboy$,2024,2026,2018,2026,2018-2026,8,9,8,...,The Thin Grey Line,71,1,11,Thy Kingdom Come,4,I Want To Die In New Orleans,9,4,91
7,$uicideboy$ x germ,$uicideBoy$ x Germ,0,0,2022,2022,2022-2022,1,0,1,...,NaN,0,0,0,DirtiestNastiest$uicide,54,DirtiestNastiest$uicide,54,2,2
8,& cardi b or nas,& Cardi B Or Nas,2019,2019,0,0,2019-2019,1,1,0,...,Rodeo,22,9,9,NaN,0,NaN,0,0,0
9,& jack harlow,& Jack Harlow,2021,2021,0,0,2021-2021,1,1,0,...,Industry Baby,1,42,42,NaN,0,NaN,0,0,0


In [21]:
df_artists.to_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/df_artists_basic.csv', index=False)
print(f"Saved df_artists to CSV with {len(df_artists)} rows")


Saved df_artists to CSV with 14226 rows
